In [1]:
import numpy as np
from numpy.random import default_rng, Generator
import trimesh
from trimesh import load, Trimesh
from trimesh.primitives import Sphere
from magnet_pinn.generator.samplers import BlobSampler, TubeSampler, PropertySampler
from magnet_pinn.generator.structures import Blob, Tube
from magnet_pinn.generator.typing import StructurePhantom
from magnet_pinn.generator.transforms import ToMesh, MeshesCleaning, Compose, MeshesTubesClipping, MeshesChildrenCutout, MeshesSequentialOperations, MeshesChildrenClipping, MeshesParentCutoutWithChildren, MeshesParentCutoutWithTubes
from magnet_pinn.generator.io import MeshWriter
from magnet_pinn.generator.structures import CustomMeshStructure
from trimesh.primitives import Sphere
from magnet_pinn.generator.utils import generate_fibonacci_points_on_sphere
from magnet_pinn.generator.phantoms import CustomPhantom, Tissue

In [2]:
cylinder_phantom = CustomPhantom(
    stl_mesh_path="./phantom.stl",
    num_children_blobs=5,
    blob_radius_decrease_per_level=0.2,
    num_tubes=10,
    relative_tube_max_radius=0.08,
    relative_tube_min_radius=0.02
)

In [3]:
print(f"Loaded mesh with {len(cylinder_phantom.parent_structure.mesh.vertices)} vertices")
print(f"Mesh bounds: {cylinder_phantom.parent_structure.mesh.bounds}")
print(f"Mesh center: {cylinder_phantom.parent_structure.position}")
print(f"Effective radius: {cylinder_phantom.parent_structure.radius:.2f}")

Loaded mesh with 100 vertices
Mesh bounds: [[-125.6217  -150.259   -138.     ]
 [ 125.6217    79.74104  138.     ]]
Mesh center: [ 5.45850530e-07 -4.16992509e+01  3.24919470e-15]
Effective radius: 190.31


In [4]:
meshes = cylinder_phantom.generate(seed=42)
print(f"Generated {len(meshes.children)} children blobs and {len(meshes.tubes)} tubes")

Attempts to place blob 2: 1
Attempts to place blob 3: 1
Attempts to place blob 4: 1
Attempts to place blob 5: 1
Attempts to place blob 6: 3
Generated 5 children blobs and 10 tubes


In [5]:
workflow = Compose([
    ToMesh(),
    MeshesSequentialOperations([
        MeshesTubesClipping(),
        MeshesChildrenClipping(),
        MeshesChildrenCutout(),
        MeshesParentCutoutWithTubes(),
        MeshesParentCutoutWithChildren()
    ])
])

In [6]:
meshes = workflow(meshes)

In [ ]:
meshes.parent.show()

In [ ]:
trimesh.boolean.union(
    meshes.tubes,
    engine='manifold'
).show()

In [ ]:
trimesh.boolean.union(
    meshes.children,
).show()

In [ ]:
trimesh.boolean.union(
    meshes.children + meshes.tubes + [meshes.parent]
).show()